In [ ]:
from thruster_allocator import thruster_positions, thruster_directions, ThrustAllocator
import matplotlib.pyplot as plt

import numpy as np
%reload_ext autoreload
%autoreload 2

# Thrust Allocator Test

## Thruster Positions and Directions

In [ ]:
%matplotlib qt
ax = plt.figure().add_subplot(projection="3d")
ax.quiver3D(*thruster_positions.T, *thruster_directions.T, length=0.1)
ax.scatter(*(0,0,0))
ax.set_aspect('equal')

## Movement Test

### Thrust Newtons

In [ ]:
%matplotlib qt
thrustAllocator = ThrustAllocator()
outputThrusts = thrustAllocator.getThrusts(
    linear_accelerations = [1.0, 0, 0],
    angular_accelerations = [0, 0, 0]
)

ax = plt.figure().add_subplot(projection="3d")
ax.quiver3D(*thruster_positions.T, *(outputThrusts[:, None] * thruster_directions).T, length=0.02)
ax.scatter(*(0,0,0))
ax.set_aspect('equal')

### Thrust PWMs

In [ ]:
pwms = thrustAllocator.getThrustPWMs(
    linear_accelerations = [0, 0, -3.0],
    angular_accelerations = [0, 0, 0]
)

pwms[np.array([3, 2, 5, 1, 4, 0])]